In [1]:
import json
import numpy as np
import os

# !pip install pywebarchive
import webarchive


import re
import json
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib
import pandas as pd

In [19]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

gestureNames = ["Forward flick", "Right flick", "Right tilt", "Left flick", "Left tilt", "Pull close", "Push away", "Turn to right", "Turn to left"]

# Import all webarchives in the directory
files = []
for r,d,f in os.walk("."):
    for file in f:
        if ".webarchive" in file and 'archivedData' not in r:
            files.append(os.path.join(r, file))


# Webarchive format -> parsed JSON dictionaries
fileData = []
for file in files:
    tmp = webarchive.open(file)
    jsonStr = cleanhtml(tmp._main_resource.data.decode())
    try:
        jsonData = json.loads(jsonStr)
        for key in jsonData.keys():
            jsonData[key] = json.loads(jsonData[key])
        fileData.append(jsonData)
    except:
        print("Failed on " + file)
print("Successfully parsed " + str(len(fileData)) + " trials")


Successfully parsed 5 trials


In [220]:
# Create list of each trial separately, tagged with user ID and everything
dataList = []

for subNum in range(len(fileData)):
    data = fileData[subNum]
    for evalName in data.keys():
        for gestureBlock in data[evalName]:
            for segment in gestureBlock:
                # Unpack each segment trial
                timestamp, detected, target, histories = segment

                detectedGest = detected[0]
                detectedGaze = detected[1]

                # Get the ground truth gesture and square out
                gestureTarget = target[0]
                gestureName = gestureNames[gestureTarget]
                gazeTarget = target[1]

                # Unpack the histories array
                headsize_hist, embeddings_hist, gazepreds_hist, IMU_hist, gestdetect_hist = histories

                dataList.append([subNum, evalName, gestureName, gazeTarget, timestamp, 
                                 gestureTarget, gazeTarget, detectedGest, detectedGaze, 
                                 headsize_hist, embeddings_hist, gazepreds_hist, gestdetect_hist, IMU_hist, 
                                ])

                
# Create dataframe out of the list
GAZELdata = pd.DataFrame(dataList, columns=['Subject', "Set", "Gesture Target Name", "Gaze Target", "Timestamp",
                                   "Target Gesture", "Target Gaze", "Detected Gesture", "Detected Gaze",
                                    "Headsize Hist", "Embeddings Hist", "Gazepreds Hist", "Gestpreds Hist", "IMU Hist"
                                   ])
# Target gestures are top to bottom
# | 1  5 |
# | 2  6 | 
# | 3  7 |
# | 4  8 |

In [282]:
# Filter the embeddings and gaze prediction histories based on movement
def getFirstActivity(lst):
    tmp = next((i for i,x in enumerate(lst) if sum(x) != 0), -1)
    # if tmp == -1: tmp = len(lst)
    return tmp


# Find index of first motion, then trim each gazepred and embedding history there
firstIndices = list(map(getFirstActivity, GAZELdata['Gestpreds Hist']))
# tmp = list(map(lambda x: x[0][:x[1]], zip(GAZELdata['Gazepreds Hist'],firstIndices)))
GAZELdata['Gazepreds Hist'] = list(map(lambda x: x[0][3:x[1]], zip(GAZELdata['Gazepreds Hist'],firstIndices)))
GAZELdata['Embeddings Hist'] = list(map(lambda x: x[0][3:x[1]], zip(GAZELdata['Embeddings Hist'],firstIndices)))

# # Plot one of the eye histories for fun
# plt.plot(GAZELdata['Gazepreds Hist'][0])

In [283]:
# Add target locations in XY
GAZELdata['Gaze Target XY'] = np.zeros((len(GAZELdata), 2)).tolist()
def seg2GridCoords(segNum):
    return [(1 + (segNum-1)//4)/3, 1/8 + (((segNum-1) % 4) )/4]
def seg2ListCoords(segNum):
    return [0.5, 1/12 + (segNum-1)/6]

grids = [x[0]=='g' for x in GAZELdata['Set']]
lists = [not x for x in grids]

GAZELdata.loc[grids, 'Gaze Target XY'] = pd.Series([seg2GridCoords(x) for x in GAZELdata.loc[grids, 'Gaze Target']])
GAZELdata.loc[lists, 'Gaze Target XY'] = pd.Series([seg2ListCoords(x) for x in GAZELdata.loc[lists, 'Gaze Target']])

In [284]:
# Train on one person's list and grid eval 1, then try it on their other two lists and grids

# Get the first evals 
subNum = 0
subData = GAZELdata.loc[GAZELdata['Subject'] == subNum]
subData

,Subject,Set,Gesture Target Name,Gaze Target,Timestamp,Target Gesture,Target Gaze,Detected Gesture,Detected Gaze,Headsize Hist,Embeddings Hist,Gazepreds Hist,Gestpreds Hist,IMU Hist,Gaze Target XY
0,0,grid1_results,Left flick,6,1615340384780,3,6,3,6.0,"[0.30224734877100423, 0.30944546044295973, 0.3...","[[0.662109375, 0.7294921875, -1.52148425579071...","[[0.5716362320117652, 0.13829771882653397], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.6464235638411402, 0.19402486358743487, -0...","[0.6666666666666666, 0.375]"
1,0,grid1_results,Left flick,4,1615340389560,3,4,3,4.0,"[0.3035961872467104, 0.3033918554893195, 0.301...","[[0.56494140625, 0.95361328125, -1.7763671875,...","[[0.46970871112174095, 0.07847121470598917], [...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.397971697522224, 0.6568024098109447, -0.3...","[0.3333333333333333, 0.875]"
2,0,grid1_results,Left flick,1,1615340398573,3,1,3,1.0,"[0.3089946410400718, 0.30877806269674807, 0.30...","[[-0.336669921875, 1.3662108182907104, -1.9765...","[[0.2608533783015937, 0.5216283193635058], [0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.7129172163193597, 0.6739144882810597, -0....","[0.3333333333333333, 0.125]"
3,0,grid1_results,Left flick,2,1615340404140,3,2,3,2.0,"[0.3143517670029907, 0.312444084606332, 0.3127...","[[-0.2509765923023224, 0.998046875, -1.5986326...","[[0.4765200522711647, 0.41857101471964575], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[-0.20187064319009035, -0.30736955834712215,...","[0.3333333333333333, 0.375]"
4,0,grid1_results,Left flick,8,1615340410342,3,8,3,8.0,"[0.30037894039833396, 0.3027644165867691, 0.30...","[[-0.5058594346046448, 1.0166015625, -1.619140...","[[0.4348910581522367, 0.5701267620692687], [0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.5471147477432704, 2.502845554836187, 2.85...","[0.6666666666666666, 0.875]"
5,0,grid1_results,Left flick,5,1615340415598,3,5,3,5.0,"[0.31187736471150646, 0.3132472158016898, 0.31...","[[-0.5029296875, 1.189453125, -1.7685546875, -...","[[0.3768615318871115, 0.7232397141783735], [0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.9427657817732767, -0.24032039270988376, -...","[0.6666666666666666, 0.125]"
6,0,grid1_results,Left flick,3,1615340422515,3,3,3,4.0,"[0.34279643728394305, 0.3431963040147383, 0.34...","[[0.1954345852136612, 1.7539061307907104, -2.3...","[[0.23675675058893286, 0.3342007970174282], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[1.1017421699650258, 2.389620228304416, 1.72...","[0.3333333333333333, 0.625]"
7,0,grid1_results,Left flick,7,1615340427697,3,7,3,8.0,"[0.3175068806548439, 0.3220397504693921, 0.318...","[[-0.73876953125, 0.7001953125, -1.30859363079...","[[0.708995730667346, 0.5265402107056778], [0.5...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[-0.05680445551138786, 0.9904426366140814, 0...","[0.6666666666666666, 0.625]"
8,0,grid1_results,Push away,8,1615340433163,6,8,6,8.0,"[0.3325384908558199, 0.3416877663270229, 0.345...","[[0.4267578125, 0.90234375, -1.639648318290710...","[[0.6245140174040626, 0.23105360467673264], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[5.698245824528633, 7.60840128725014, 7.9918...","[0.6666666666666666, 0.875]"
9,0,grid1_results,Push away,3,1615340440229,6,3,6,4.0,"[0.3913670235294598, 0.3958911987480109, 0.395...","[[-0.1896972805261612, 1.839843511581421, -2.7...","[[0.41092213652803206, 0.42267938816411266], [...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.0683325875307529, 0.524683712817861, 0.06...","[0.3333333333333333, 0.625]"


In [285]:
GAZELdata.head()

,Subject,Set,Gesture Target Name,Gaze Target,Timestamp,Target Gesture,Target Gaze,Detected Gesture,Detected Gaze,Headsize Hist,Embeddings Hist,Gazepreds Hist,Gestpreds Hist,IMU Hist,Gaze Target XY
0,0,grid1_results,Left flick,6,1615340384780,3,6,3,6.0,"[0.30224734877100423, 0.30944546044295973, 0.3...","[[0.662109375, 0.7294921875, -1.52148425579071...","[[0.5716362320117652, 0.13829771882653397], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.6464235638411402, 0.19402486358743487, -0...","[0.6666666666666666, 0.375]"
1,0,grid1_results,Left flick,4,1615340389560,3,4,3,4.0,"[0.3035961872467104, 0.3033918554893195, 0.301...","[[0.56494140625, 0.95361328125, -1.7763671875,...","[[0.46970871112174095, 0.07847121470598917], [...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.397971697522224, 0.6568024098109447, -0.3...","[0.3333333333333333, 0.875]"
2,0,grid1_results,Left flick,1,1615340398573,3,1,3,1.0,"[0.3089946410400718, 0.30877806269674807, 0.30...","[[-0.336669921875, 1.3662108182907104, -1.9765...","[[0.2608533783015937, 0.5216283193635058], [0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.7129172163193597, 0.6739144882810597, -0....","[0.3333333333333333, 0.125]"
3,0,grid1_results,Left flick,2,1615340404140,3,2,3,2.0,"[0.3143517670029907, 0.312444084606332, 0.3127...","[[-0.2509765923023224, 0.998046875, -1.5986326...","[[0.4765200522711647, 0.41857101471964575], [0...","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[-0.20187064319009035, -0.30736955834712215,...","[0.3333333333333333, 0.375]"
4,0,grid1_results,Left flick,8,1615340410342,3,8,3,8.0,"[0.30037894039833396, 0.3027644165867691, 0.30...","[[-0.5058594346046448, 1.0166015625, -1.619140...","[[0.4348910581522367, 0.5701267620692687], [0....","[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0,...","[[[0.5471147477432704, 2.502845554836187, 2.85...","[0.6666666666666666, 0.875]"


In [151]:
True & True

True